In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=phenotypes.columns.difference(['age','sex','cohort','diabetes'])

In [4]:
len(binary_continuous_traits)

97

In [5]:
plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('AA_')]

,chrom,pos,cm,a1,a2
snp,,,,,
AA_A_-15_29910359_exon1,6,29910359,0,L,V
AA_A_-11_29910371_exon1,6,29910371,0,L,S
AA_A_9_29910558_exon2_F,6,29910558,0,p,a
AA_A_9_29910558_exon2_S,6,29910558,0,p,a
AA_A_9_29910558_exon2_T,6,29910558,0,p,a
...,...,...,...,...,...
AA_DPB1_170_33052900_exon3_T,6,33052900,0,p,a
AA_DPB1_178_33052924_exon3_L,6,33052924,0,p,a
AA_DPB1_178_33052924_exon3_M,6,33052924,0,p,a


In [6]:
def get_a1_freq_case_control(row,binary=True):
    
    if row['marker_name'] not in plink_KCHIP_HLA_AA_SNP_1000G_bim.index.values:
        return np.nan,np.nan
    
    dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(row['marker_name']).astype(float)
    dosage[dosage==-1]=np.nan
    
    if plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A1'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A2']:
        pass
    elif plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A2'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A1']:
        dosage=2-dosage

    if binary:
        dosage_control=dosage[pheno['pheno']==1]
        dosage_case=dosage[pheno['pheno']==2]

        a1_freq_case=(2*(dosage_case==2).sum()+1*(dosage_case==1).sum())/(2*((~np.isnan(dosage_case)).sum()))
        a1_freq_control=(2*(dosage_control==2).sum()+1*(dosage_control==1).sum())/(2*((~np.isnan(dosage_control)).sum()))

        return a1_freq_case,a1_freq_control
    else:
        dosage_case=dosage[pheno['pheno']!=-9]

        a1_freq_case=(2*(dosage_case==2).sum()+1*(dosage_case==1).sum())/(2*((~np.isnan(dosage_case)).sum()))

        return a1_freq_case,np.nan

In [7]:
def name_to_pos(name):
    if name[:3]=='AA_':
        return int(name.split('_')[3])
    elif name[:4]=='HLA_':
        return plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_'+name.split('*')[0].split('_')[1]+'*')].iloc[0]['pos']
    else:
        raise

In [8]:
SNP_DB=pd.read_csv('data/mart_export_SNP_grch37.txt',sep='\t')

In [13]:
SNP_DB.set_index('Variant name').loc['rs9281747']

Variant source                                                                         dbSNP
Chromosome/scaffold name                                                                   6
Chromosome/scaffold position start (bp)                                             32324082
Chromosome/scaffold position end (bp)                                               32324090
Variant alleles                            TTTTTTTTT/TTTTTTTT/TTTTTTTTTT/TTTTTTTTTTT/TTTT...
Name: rs9281747, dtype: object

In [12]:
SNP_DB.set_index('Variant name').loc['rs9281747']['Variant alleles']

'TTTTTTTTT/TTTTTTTT/TTTTTTTTTT/TTTTTTTTTTT/TTTTTTTTTTTTTT'

In [11]:
SNP_DB.set_index('Variant name').loc['rs371860917']

KeyError: 'rs371860917'

In [8]:
SNP_DB.head()

,Variant name,Variant source,Chromosome/scaffold name,Chromosome/scaffold position start (bp),Chromosome/scaffold position end (bp),Variant alleles
0,rs1251583312,dbSNP,6,28000000,28000000,T/C
1,rs978125173,dbSNP,6,28000007,28000007,C/G
2,rs1212093726,dbSNP,6,28000010,28000010,G/A
3,rs924179820,dbSNP,6,28000012,28000012,A/G
4,rs966310339,dbSNP,6,28000021,28000021,A/G


In [15]:
SNP_DB['Variant alleles set']=SNP_DB['Variant alleles'].apply(lambda x: set(x.split('/')).difference(['-']))

In [16]:
rsid_mapper=plink_KCHIP_HLA_AA_SNP_1000G_bim.copy()
rsid_mapper=rsid_mapper[(~rsid_mapper.index.str.contains('HLA'))
                        |~(rsid_mapper.index.str.contains('AA_'))]

In [17]:
rsid_mapper

,chrom,pos,cm,a1,a2
snp,,,,,
6:28000361_T/C,6,28000361,0,C,T
6:28000610_C/A,6,28000610,0,A,C
6:28001003_A/C,6,28001003,0,C,A
6:28001033_T/C,6,28001033,0,C,T
6:28001610_T/C,6,28001610,0,C,T
...,...,...,...,...,...
6:34997188_C/T,6,34997188,0,T,C
6:34997259_G/A,6,34997259,0,A,G
6:34997606_T/A,6,34997606,0,A,T


In [21]:
pos_a1_a2_to_rsid(rsid_mapper.loc['SNPS_DRB1_9798_32547722_intron5_A'])

'SNPS_DRB1_9798_32547722_intron5_A'

In [18]:
def pos_a1_a2_to_rsid(row):
    
    SNP_DB_match=SNP_DB[(SNP_DB['Chromosome/scaffold position start (bp)']<=row['pos'])&
                        (SNP_DB['Chromosome/scaffold position end (bp)']>=row['pos'])]
    if len(SNP_DB_match)==0:
        return row.name
    
    SNP_DB_match_match=SNP_DB_match[SNP_DB_match['Variant alleles set'].apply(lambda x: set([row['a1'],row['a2']]).issubset(x))]
    if len(SNP_DB_match_match)==0:
        return row.name
    elif len(SNP_DB_match_match)==1:
        return SNP_DB_match_match.iloc[0]['Variant name']

    SNP_DB_match_match_match=SNP_DB_match_match[(SNP_DB_match_match['Chromosome/scaffold position start (bp)']<=row['pos'])&
                            (SNP_DB_match_match['Chromosome/scaffold position end (bp)']>=row['pos'])]        
    if len(SNP_DB_match_match_match)==0:
        return SNP_DB_match_match.iloc[0]['Variant name']
    elif len(SNP_DB_match_match_match)==1:
        return SNP_DB_match_match_match.iloc[0]['Variant name']
    else:
        SNP_DB_match_match_match.iloc[0]['Variant name']

In [19]:
rsid_mapper['marker_name_pub']=rsid_mapper.apply(pos_a1_a2_to_rsid,axis=1)

KeyboardInterrupt: 

In [44]:
def marker_name_to_marker_name_pub(marker_name):
    if marker_name[:4]=='HLA_':
        return marker_name.replace('_','-')
    elif marker_name[:3]=='AA_':
        marker_name_split=marker_name.split('_')
        if len(marker_name_split)==5:
            return 'HLA-{} pos.{}'.format(marker_name_split[1],marker_name_split[2])
        elif len(marker_name_split)==6:
            return 'HLA-{} pos.{}({})'.format(marker_name_split[1],marker_name_split[2],marker_name_split[-1])
    else:
        return rsid_mapper.loc[marker_name]['marker_name_pub']

In [59]:
rsid_mapper[rsid_mapper['pos']==32547722]

,chrom,pos,cm,a1,a2,marker_name_pub
snp,,,,,,
SNPS_DRB1_9798_32547722_intron5_A,6,32547722,0,p,a,SNPS_DRB1_9798_32547722_intron5_A
SNPS_DRB1_9798_32547722_intron5_C,6,32547722,0,p,a,SNPS_DRB1_9798_32547722_intron5_C
SNPS_DRB1_9798_32547722_intron5_T,6,32547722,0,p,a,SNPS_DRB1_9798_32547722_intron5_T
SNPS_DRB1_9798_32547722_intron5_x,6,32547722,0,p,a,SNPS_DRB1_9798_32547722_intron5_x


In [62]:
rsid_mapper[rsid_mapper['pos']==31324530]

,chrom,pos,cm,a1,a2,marker_name_pub
snp,,,,,,
AA_B_69_31324530_exon2_A,6,31324530,0,p,a,AA_B_69_31324530_exon2_A
AA_B_69_31324530_exon2_R,6,31324530,0,p,a,AA_B_69_31324530_exon2_R
AA_B_69_31324530_exon2_T,6,31324530,0,p,a,AA_B_69_31324530_exon2_T
SNPS_B_406_31324530_exon2,6,31324530,0,C,G,rs41546313


In [50]:
len(binary_continuous_traits)

97

In [57]:
#GAT_result=
GAT_result[((~GAT_result['marker_name'].str.contains('HLA',regex=False))|(GAT_result['marker_name'].str.contains('*',regex=False)))]

,marker_name,P,nobs,coef,std,Z,chisq,df,term,A1,A2,multi_allele,note,POS
0,AA_A_-11_29910371_exon1,0.448845,157320,0.020633,0.027244,0.757342,NaN,NaN,THIS,L,S,NaN,phased biallelic,29910371
1,AA_A_-15_29910359_exon1,0.333962,157320,-0.022524,0.023312,-0.966165,NaN,NaN,THIS,L,V,NaN,phased biallelic,29910359
2,AA_A_105_29911087_exon3,0.505438,157320,-0.019221,0.028863,-0.665958,NaN,NaN,THIS,P,S,NaN,phased biallelic,29911087
3,AA_A_107_29911093_exon3,0.723441,157320,-0.008895,0.025136,-0.353863,NaN,NaN,THIS,W,G,NaN,phased biallelic,29911093
4,AA_A_114_29911114_exon3,0.312846,157320,NaN,NaN,NaN,3.561713,3.0,NaN,NaN,NaN,"AA_A_114_29911114_exon3_E,AA_A_114_29911114_ex...",phased multiallelic,29911114
5,AA_A_114_29911114_exon3_E,0.346732,157320,-0.055250,0.058717,-0.940948,NaN,NaN,THIS,p,a,NaN,phased biallelic,29911114
6,AA_A_114_29911114_exon3_H,0.767843,157320,-0.006759,0.022896,-0.295197,NaN,NaN,THIS,a,p,NaN,phased biallelic,29911114
7,AA_A_114_29911114_exon3_Q,0.236482,157320,0.029773,0.025150,1.183826,NaN,NaN,THIS,p,a,NaN,phased biallelic,29911114
8,AA_A_114_29911114_exon3_R,0.164096,157320,-0.045255,0.032524,-1.391426,NaN,NaN,THIS,p,a,NaN,phased biallelic,29911114
9,AA_A_116_29911120_exon3,0.788450,157320,NaN,NaN,NaN,1.052917,3.0,NaN,NaN,NaN,"AA_A_116_29911120_exon3_D,AA_A_116_29911120_ex...",phased multiallelic,29911120


In [54]:
GAT_result

,marker_name,P,nobs,coef,std,Z,chisq,df,term,A1,A2,multi_allele,note,POS
0,AA_A_-11_29910371_exon1,0.448845,157320,0.020633,0.027244,0.757342,NaN,NaN,THIS,L,S,NaN,phased biallelic,29910371
1,AA_A_-15_29910359_exon1,0.333962,157320,-0.022524,0.023312,-0.966165,NaN,NaN,THIS,L,V,NaN,phased biallelic,29910359
2,AA_A_105_29911087_exon3,0.505438,157320,-0.019221,0.028863,-0.665958,NaN,NaN,THIS,P,S,NaN,phased biallelic,29911087
3,AA_A_107_29911093_exon3,0.723441,157320,-0.008895,0.025136,-0.353863,NaN,NaN,THIS,W,G,NaN,phased biallelic,29911093
4,AA_A_114_29911114_exon3,0.312846,157320,NaN,NaN,NaN,3.561713,3.0,NaN,NaN,NaN,"AA_A_114_29911114_exon3_E,AA_A_114_29911114_ex...",phased multiallelic,29911114
5,AA_A_114_29911114_exon3_E,0.346732,157320,-0.055250,0.058717,-0.940948,NaN,NaN,THIS,p,a,NaN,phased biallelic,29911114
6,AA_A_114_29911114_exon3_H,0.767843,157320,-0.006759,0.022896,-0.295197,NaN,NaN,THIS,a,p,NaN,phased biallelic,29911114
7,AA_A_114_29911114_exon3_Q,0.236482,157320,0.029773,0.025150,1.183826,NaN,NaN,THIS,p,a,NaN,phased biallelic,29911114
8,AA_A_114_29911114_exon3_R,0.164096,157320,-0.045255,0.032524,-1.391426,NaN,NaN,THIS,p,a,NaN,phased biallelic,29911114
9,AA_A_116_29911120_exon3,0.788450,157320,NaN,NaN,NaN,1.052917,3.0,NaN,NaN,NaN,"AA_A_116_29911120_exon3_D,AA_A_116_29911120_ex...",phased multiallelic,29911120


In [ ]:
for phenotype_name in binary_continuous_traits[0:30]:
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
    
    phenotype_type='binary' if len(pheno['pheno'][pheno['pheno']!=-9].value_counts())<3 else 'continuous'    
    print(phenotype_name,phenotype_type)
    try:
        for step_idx_sub in range(1,100):
            print(step_idx_sub,end=' ')
            if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
                print('meets end',step_idx_sub)
                break

            GAT_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.GAT.result.tsv'.format(step_idx_sub),sep='\t')
            
            #GAT_result[((~GAT_result['marker_name'].str.contains('HLA',regex=False))|(GAT_result['marker_name'].str.contains('*',regex=False)))]
            
            GAT_result['POS']=GAT_result['marker_name'].map(name_to_pos)
            #GAT_result=GAT_result[(~GAT_result['marker_name'].str.contains('HLA',regex=False))|(GAT_result['marker_name'].str.contains('*',regex=False))]
            if step_idx_sub>1:
                conditional_list_used=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub),sep='\t',header=None)[0].values.tolist()
                for condition in conditional_list_used:
                    if condition[:4]=='HLA_':
                        GAT_result.loc[GAT_result['marker_name'].str.contains('AA_'+condition.split('_')[1]+'_',regex=False), ['P', 'coef', 'std', 'Z', 'chisq', 'df']]=np.nan            

            plink_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(step_idx_sub,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t')
            plink_result_munge=plink_result[plink_result['TEST']=='ADD'].drop(columns='#CHROM').rename(columns={'TEST':'term','ID':'marker_name','BETA':'coef','SE':'std','LOG(OR)_SE':'std','T_STAT':'Z','Z_STAT':'Z','OBS_CT':'nobs'})
            
            if 'OR' in plink_result_munge.columns:
                plink_result_munge['coef']=np.log(plink_result_munge['OR'])            
            
            plink_result_munge['A2']=plink_result_munge.apply(lambda x: x['ALT'] if x['A1']==x['REF'] else x['REF'],axis=1)
            plink_result_munge=plink_result_munge.drop(columns=['REF','ALT'])
            plink_result_munge['note']='unphased bialleic'
            
            
            result_merge=pd.concat([GAT_result,plink_result_munge],sort=True)
            
            
            
            if phenotype_type=='binary':
                assert set(pheno['pheno'].unique())=={-9,1,2}
                
                
            a1_freq_case_control=result_merge.apply(lambda row: get_a1_freq_case_control(row,binary=(True if phenotype_type=='binary' else False)),axis=1)
            result_merge['A1_freq_case']=[i[0] for i in a1_freq_case_control]
            result_merge['A1_freq_control']=[i[1] for i in a1_freq_case_control]            
            
            
            result_merge['phenotype_name']=phenotype_name
            result_merge['phenotype_type']=phenotype_type
            
            result_merge['marker_name_pub']=result_merge['marker_name'].apply(marker_name_to_marker_name_pub)
            
            if phenotype_type=='binary':
                result_merge['samples(case/control)']='{}/{}'.format((pheno['pheno']==2).sum(),(pheno['pheno']==1).sum())
            else:
                result_merge['samples(case/control)']='{}'.format((pheno['pheno']!=-9).sum())

            result_merge['step']=step_idx_sub
            with open(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub)) as f:
                result_merge['condition']=','.join(f.read().split())        
            result_merge=result_merge[['phenotype_type','phenotype_name','samples(case/control)','step','condition','marker_name','marker_name_pub','note','term','POS','A1','A2','A1_freq_case','A1_freq_control','multi_allele','nobs','Z','coef','std','chisq','df','P']]
            result_merge.sort_values('POS').to_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')
    except:
        print('error',phenotype_name,step_idx_sub)
        
    

FEV_over_FVC_predicted continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars


2 meets end 2
FEV_predicted continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars


2 

...

2 meets end 2
breast_cancer binary
1 2 meets end 2
bun continuous
1 2 3 meets end 3
cataract binary
1 2 meets end 2
cervical_cancer binary
1 2 meets end 2
child_dead continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars


2 meets end 2
child_delivery_number continuous
1 2 meets end 2
child_present continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in long_scalars


2 meets end 2
cholelithiasis_cholecystitis binary
1 2 meets end 2
chronic_bronchitis binary
1 2 meets end 2
chronic_gastritis_stomach_ulcer binary
1 2 meets end 2
colon_polyps binary
1 2 meets end 2


# deprecated

In [49]:
#result_merge

In [52]:
for phenotype_name in binary_continuous_traits[0:30]:
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
    
    phenotype_type='binary' if len(pheno['pheno'][pheno['pheno']!=-9].value_counts())<3 else 'continuous'    
    print(phenotype_name,phenotype_type)
    
    for step_idx_sub in range(1,100):
        print(step_idx_sub,end=' ')
        if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
            print('meets end',step_idx_sub)
            break

        GAT_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.GAT.result.tsv'.format(step_idx_sub),sep='\t')
        GAT_result['POS']=GAT_result['marker_name'].map(name_to_pos)
        #GAT_result=GAT_result[(~GAT_result['marker_name'].str.contains('HLA',regex=False))|(GAT_result['marker_name'].str.contains('*',regex=False))]
        if step_idx_sub>1:
            conditional_list_used=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub),sep='\t',header=None)[0].values.tolist()
            for condition in conditional_list_used:
                if condition[:4]=='HLA_':
                    GAT_result.loc[GAT_result['marker_name'].str.contains('AA_'+condition.split('_')[1]+'_',regex=False), ['P', 'coef', 'std', 'Z', 'chisq', 'df']]=np.nan            

        plink_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(step_idx_sub,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t')
        plink_result_munge=plink_result[plink_result['TEST']=='ADD'].drop(columns='#CHROM').rename(columns={'TEST':'term','ID':'marker_name','BETA':'coef','SE':'std','LOG(OR)_SE':'std','T_STAT':'Z','Z_STAT':'Z','OBS_CT':'nobs'})

        if 'OR' in plink_result_munge.columns:
            plink_result_munge['coef']=np.log(plink_result_munge['OR'])            

        plink_result_munge['A2']=plink_result_munge.apply(lambda x: x['ALT'] if x['A1']==x['REF'] else x['REF'],axis=1)
        plink_result_munge=plink_result_munge.drop(columns=['REF','ALT'])
        plink_result_munge['note']='unphased bialleic'


        result_merge=pd.concat([GAT_result,plink_result_munge],sort=True)



        if phenotype_type=='binary':
            assert set(pheno['pheno'].unique())=={-9,1,2}


        a1_freq_case_control=result_munge.apply(lambda row: get_a1_freq_case_control(row,binary=(True if phenotype_type=='binary' else False)),axis=1)
        result_munge['A1_freq_case']=[i[0] for i in a1_freq_case_control]
        result_munge['A1_freq_control']=[i[1] for i in a1_freq_case_control]            


        result_merge['phenotype_name']=phenotype_name
        result_merge['phenotype_type']=phenotype_type

        result_merge['marker_name_pub']=result_merge['marker_name'].apply(marker_name_to_marker_name_pub)

        if phenotype_type=='binary':
            result_merge['samples(case/control)']='{}/{}'.format((pheno['pheno']==2).sum(),(pheno['pheno']==1).sum())
        else:
            result_merge['samples(case/control)']='{}'.format((pheno['pheno']!=-9).sum())

        result_merge['step']=step_idx_sub
        with open(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub)) as f:
            result_merge['condition']=','.join(f.read().split())        
        result_merge=result_merge[['phenotype_type','phenotype_name','samples(case/control)','step','condition','marker_name','marker_name_pub','note','term','POS','A1','A2','A1_freq_case','A1_freq_control','multi_allele','nobs','Z','coef','std','chisq','df','P']]
        result_merge.sort_values('POS').to_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')


    

FEV_over_FVC_predicted continuous
1 

NameError: name 'result_munge' is not defined

In [32]:
result_merge

,A1,A2,P,POS,Z,chisq,coef,df,marker_name,multi_allele,nobs,note,std,term,A1_freq_case,A1_freq_control
0,L,S,0.0664257,29910371,-1.835540,NaN,-0.022526,NaN,AA_A_-11_29910371_exon1,NaN,14880,phased biallelic,0.012272,THIS,0.222715,NaN
1,L,V,0.0867021,29910359,-1.713057,NaN,-0.017739,NaN,AA_A_-15_29910359_exon1,NaN,14880,phased biallelic,0.010355,THIS,0.417339,NaN
2,P,S,0.0503331,29911087,1.957122,NaN,0.025112,NaN,AA_A_105_29911087_exon3,NaN,14880,phased biallelic,0.012831,THIS,0.197312,NaN
3,W,G,0.978276,29911093,0.027231,NaN,0.000306,NaN,AA_A_107_29911093_exon3,NaN,14880,phased biallelic,0.011229,THIS,0.292406,NaN
4,NaN,NaN,0.531223,29911114,NaN,2.203685,NaN,3.0,AA_A_114_29911114_exon3,"AA_A_114_29911114_exon3_E,AA_A_114_29911114_ex...",14880,phased multiallelic,NaN,NaN,NaN,NaN
5,p,a,0.202712,29911114,-1.273864,NaN,-0.030807,NaN,AA_A_114_29911114_exon3_E,NaN,14880,phased biallelic,0.024184,THIS,0.046774,NaN
6,a,p,0.473773,29911114,-0.716354,NaN,-0.007321,NaN,AA_A_114_29911114_exon3_H,NaN,14880,phased biallelic,0.010219,THIS,0.482997,NaN
7,p,a,0.570792,29911114,-0.566885,NaN,-0.006441,NaN,AA_A_114_29911114_exon3_Q,NaN,14880,phased biallelic,0.011363,THIS,0.280847,NaN
8,p,a,0.647274,29911114,0.457553,NaN,0.006450,NaN,AA_A_114_29911114_exon3_R,NaN,14880,phased biallelic,0.014096,THIS,0.155376,NaN
9,NaN,NaN,0.381169,29911120,NaN,3.068602,NaN,3.0,AA_A_116_29911120_exon3,"AA_A_116_29911120_exon3_D,AA_A_116_29911120_ex...",14880,phased multiallelic,NaN,NaN,NaN,NaN


In [ ]:
for phenotype_name in binary_continuous_traits:
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
    
    phenotype_type='binary' if len(pheno['pheno'][pheno['pheno']!=-9].value_counts())<3 else 'continuous'    
    print(phenotype_name,phenotype_type)

    for step_idx_sub in range(1,100):
        print(step_idx_sub,end=' ')
        if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
            print('meets end',step_idx_sub)
            break

        result_temp=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.mergetemp.result.tsv'.format(step_idx_sub),sep='\t').set_index('marker_name')
        print(result_temp.head())
        GAT_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.GAT.result.tsv'.format(step_idx_sub),sep='\t')
        GAT_result['POS']=GAT_result['marker_name'].map(name_to_pos)
        #GAT_result=GAT_result[(~GAT_result['marker_name'].str.contains('HLA',regex=False))|(GAT_result['marker_name'].str.contains('*',regex=False))]
        if step_idx_sub>1:
            conditional_list_used=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub),sep='\t',header=None)[0].values.tolist()
            for condition in conditional_list_used:
                if condition[:4]=='HLA_':
                    GAT_result.loc[GAT_result['marker_name'].str.contains('AA_'+condition.split('_')[1]+'_',regex=False), ['P', 'coef', 'std', 'Z', 'chisq', 'df']]=np.nan            

        plink_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(step_idx_sub,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t')
        plink_result_munge=plink_result[plink_result['TEST']=='ADD'].drop(columns='#CHROM').rename(columns={'TEST':'term','ID':'marker_name','BETA':'coef','SE':'std','LOG(OR)_SE':'std','T_STAT':'Z','Z_STAT':'Z','OBS_CT':'nobs'})

        if 'OR' in plink_result_munge.columns:
            plink_result_munge['coef']=np.log(plink_result_munge['OR'])            

        plink_result_munge['A2']=plink_result_munge.apply(lambda x: x['ALT'] if x['A1']==x['REF'] else x['REF'],axis=1)

        if phenotype_type=='binary':
            assert set(pheno['pheno'].unique())=={-9,1,2}
            a1_freq_case_control=plink_result_munge.apply(get_a1_freq_case_control,axis=1)
            plink_result_munge['A1_freq_case']=[i[0] for i in a1_freq_case_control]
            plink_result_munge['A1_freq_control']=[i[1] for i in a1_freq_case_control]
        else:
            a1_freq_case_nan=plink_result_munge.apply(get_a1_freq_case_nan,axis=1)
            plink_result_munge['A1_freq_case']=[i[0] for i in a1_freq_case_nan]
            plink_result_munge['A1_freq_control']=np.nan

        plink_result_munge=plink_result_munge.drop(columns=['REF','ALT'])
        plink_result_munge['note']='unphased bialleic'

        result_merge=pd.concat([GAT_result,plink_result_munge],sort=True)
        result_merge['phenotype_name']=phenotype_name
        result_merge['phenotype_type']=phenotype_type

        result_merge['marker_name_pub']=result_merge['marker_name'].apply(marker_name_to_marker_name_pub)

        if phenotype_type=='binary':
            result_merge['samples(case/control)']='{}/{}'.format((pheno['pheno']==2).sum(),(pheno['pheno']==1).sum())
        else:
            result_merge['samples(case/control)']='{}'.format((pheno['pheno']!=-9).sum())

        result_merge['step']=step_idx_sub
        with open(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub)) as f:
            result_merge['condition']=','.join(f.read().split())        
        result_merge=result_merge[['phenotype_type','phenotype_name','samples(case/control)','step','condition','marker_name','marker_name_pub','note','term','POS','A1','A2','A1_freq_case','A1_freq_control','multi_allele','nobs','Z','coef','std','chisq','df','P']]
        #result_merge.sort_values('POS').to_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')
    break

In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

from basic_tools import *

"""
jupyter nbconvert 5_2_report_generation.ipynb --to script

for i in {00..20};do python 5_2_report_generation.py $i;done
for i in {21..40};do python 5_2_report_generation.py $i;done
for i in {41..60};do python 5_2_report_generation.py $i;done
for i in {61..80};do python 5_2_report_generation.py $i;done
for i in {81..98};do python 5_2_report_generation.py $i;done
"""

'\njupyter nbconvert 5_2_report_generation.ipynb --to script\n\nfor i in {00..20};do python 5_2_report_generation.py $i;done\nfor i in {21..40};do python 5_2_report_generation.py $i;done\nfor i in {41..60};do python 5_2_report_generation.py $i;done\nfor i in {61..80};do python 5_2_report_generation.py $i;done\nfor i in {81..98};do python 5_2_report_generation.py $i;done\n'

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=phenotypes.columns.difference(['age','sex','cohort','diabetes'])
print(len(binary_continuous_traits))

if 'ipykernel' in sys.argv[0]:
    ipykernel=True
    phenotype_name='thyroid_disease'
    #phenotype_name='height'
else:
    ipykernel=False
    phenotype_name=sys.argv[1]
    
if phenotype_name.isdigit():
    phenotype_name=int(phenotype_name)
    phenotype_name=binary_continuous_traits[phenotype_name]      

97


In [4]:
data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
pathlib.Path(data_out_assoc_phenotype_path).mkdir(parents=True, exist_ok=True)

In [5]:
pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
phenotype_type='binary' if len(pheno['pheno'][pheno['pheno']!=-9].value_counts())<3 else 'continuous'
phenotype_type

'binary'

In [6]:
log = logging.getLogger('logger')
log.setLevel(logging.DEBUG)

log_file_name=datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+'.log'
log_file_path=data_out_assoc_phenotype_path+log_file_name
fileHandler = logging.FileHandler(log_file_path)
streamHandler = logging.StreamHandler()

formatter = logging.Formatter('%(message)s')
fileHandler.setFormatter(formatter)
streamHandler.setFormatter(formatter)

log.addHandler(fileHandler)
log.addHandler(streamHandler)

In [7]:
log.info_head=lambda x: log.info('-'*int((100-len(x))/2)+x+'-'*int((100-len(x))/2))

In [8]:
def get_a1_freq_case_control(row):
    if 'bialle' in row['note']:
        dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(row['marker_name']).astype(float)
        dosage[dosage==-1]=np.nan
        if plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A1'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A2']:
            pass
        elif plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A2'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A1']:
            dosage=2-dosage

        dosage_control=dosage[pheno['pheno']==1]
        dosage_case=dosage[pheno['pheno']==2]

        a1_freq_case=(2*(dosage_case==2).sum()+1*(dosage_case==1).sum())/(2*(dosage_case!=-1).sum())
        a1_freq_control=(2*(dosage_control==2).sum()+1*(dosage_control==1).sum())/(2*(dosage_control!=-1).sum())

        return a1_freq_case,a1_freq_control
    else:
        return np.nan,np.nan

In [9]:
def name_to_pos(name):
    if name[:3]=='AA_':
        return int(name.split('_')[3])
    elif name[:4]=='HLA_':
        return plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_'+name.split('*')[0].split('_')[1]+'*',regex=False)].iloc[0]['pos']
    else:
        raise

In [ ]:
for step_idx_sub in range(1,100):
    log.info_head("phenotype_name: {}, phenotype_type:{} , Step : {} ".format(phenotype_name,phenotype_type,step_idx_sub))
    
    if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
        log.info("step_{}.cond.stop->stops ".format(step_idx_sub))
        sys.exit()
    
    result_merge=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.result.tsv'.format(step_idx_sub),sep='\t')                                        
    result_merge['note']=result_merge['note'].str.replace('phased bialleic','phased biallelic')
    
    result_merge['POS'][result_merge['POS'].isnull()]=result_merge[result_merge['POS'].isnull()]['marker_name'].apply(name_to_pos)
    assert len(result_merge[result_merge['POS'].isnull()])==0

    result_merge['phenotype_name']=phenotype_name
    result_merge['step']=step_idx_sub    

    with open(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub)) as f:
        result_merge['condition']=','.join(f.read().split())      
        
        
    if phenotype_type=='binary':
        assert set(pheno['pheno'].unique())=={-9,1,2}
        if step_idx_sub==1:
            a1_freq_case_control=result_merge.apply(get_a1_freq_case_control,axis=1)
        else:
            assert np.all(result_merge_save['phenotype_name']==phenotype_name)
            assert np.all(result_merge_save['marker_name']==result_merge['marker_name'])
            assert np.all(result_merge_save['A1'].fillna(0)==result_merge['A1'].fillna(0))
            
        result_merge['A1_freq_case']=[i[0] for i in a1_freq_case_control]
        result_merge['A1_freq_control']=[i[1] for i in a1_freq_case_control]

        result_merge['samples(case/control)']='{}/{}'.format((pheno['pheno']==2).sum(),(pheno['pheno']==1).sum())
    else:
        result_merge['A1_freq_case'],result_merge['A1_freq_control']=np.nan,np.nan

        result_merge['samples(case/control)']='{}'.format((pheno['pheno']!=-9).sum())
        
        
    result_merge_save=result_merge[['phenotype_name','samples(case/control)','step','condition','marker_name','note','term','POS','A1','A2','A1_freq_case','A1_freq_control','multi_allele','Z','coef','std','chisq','df','nobs','P']]
    result_merge_save.sort_values('POS').to_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t',index=None)        